In [4]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter
from IAP_seqs import Seq

# Lista Cenas Importantes

Erro mt random a povoar references

Ver parte das sequences --> Precisamos de mais espaço mas o SQL limita isso  Data too long for column 'Translacion_seq' at row 1

Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`Genbank`, CONSTRAINT `Genbank-references` FOREIGN KEY (`ID`) REFERENCES `References` (`ID`))

Ligar Base de Dados

In [32]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()


Ficheiro GB 

In [6]:
id_genbank_input = input("ID NCBI")

In [7]:
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb") # dowload do ficheiro
record = SeqIO.read(handle, "gb")

/home/sa_bruno/anaconda3/envs/mestrado_final/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


In [8]:
# # file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

# Obter todas as variáveis / Povoar Base de Dados 

# Tabela Locus

In [9]:
# Locus 
ID = record.name
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [10]:
print(ID)
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

NC_003539
284
ss-RNA circular
VRL
13-AUG-2018


In [11]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

# Tabela Genbank

Usamos


In [12]:
# Genbank
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""

for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [13]:
print(ID)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

NC_003539
Citrus bark cracking viroid
NC_003539
Citrus viroid IV virus, complete genome
RefSeq, genome, viroid RNA
CTGGGGAATTTCTCTGCGGGACCAAATAAAAACAGCTTGTGGAGGGAACATACCTGAAGAGGGATCCCCGGGGAAATCTCTTCAGACTCGTCGAGGGGAGGGCGCCGCGGATCACTGGCGTCCAGCACCGGAACAGGAGCTCGTCTCCTTCCTTCCATCGCTGGCTCCACATCCGATCGTCGCTTCTTCCTTCGCGACCTGAGATAGAAACTACCCGGTGGATACAACTCTTGGGTTGTTCCTCCCAGGCTTGTTAATAAAAATGGCCCGCGTTTGAGACCCCT


In [14]:
try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID, Organism, Accession, Definition, Keywords)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                        (ID, Organism, Accession, Definition, Keywords))
except:
        print("Erro Povoação Genbank")

Erro Povoação Genbank


# Tabela Sequences  / Parte para Algoritmos

In [15]:
alg_seq = Seq(Sequence).count_nucleotides()

Count_A = alg_seq[0]
Count_C = alg_seq[1]
Count_G = alg_seq[2]
Count_T = alg_seq[3]

In [16]:
print(ID)
print(Sequence)
print(Count_A)
print(Count_C)
print(Count_G)
print(Count_T)

NC_003539
CTGGGGAATTTCTCTGCGGGACCAAATAAAAACAGCTTGTGGAGGGAACATACCTGAAGAGGGATCCCCGGGGAAATCTCTTCAGACTCGTCGAGGGGAGGGCGCCGCGGATCACTGGCGTCCAGCACCGGAACAGGAGCTCGTCTCCTTCCTTCCATCGCTGGCTCCACATCCGATCGTCGCTTCTTCCTTCGCGACCTGAGATAGAAACTACCCGGTGGATACAACTCTTGGGTTGTTCCTCCCAGGCTTGTTAATAAAAATGGCCCGCGTTTGAGACCCCT
62
82
76
64


In [17]:
try:

        cur.execute("""
                INSERT INTO Sequences (ID, Sequence, Count_A, Count_C, Count_T, Count_G)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (ID, Sequence, Count_A, Count_C, Count_T, Count_G))   

except:
        print("Erro Povoação Sequences")

Erro Povoação Sequences


# Tabela Features

In [18]:
temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
Location_span = str(record.features[0].location)

In [19]:
print(ID)
print(CDS_count)
print(Gene)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)

NC_003539
0
0
0
0
4
0
[0:284](+)


In [20]:
try:
        cur.execute("""
        INSERT INTO Features (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 
except:
        print("Erro Features")

Erro Features


In [21]:
cur.execute("""
        INSERT INTO Features (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 

ProgrammingError: 1054 (42S22): Unknown column 'ID' in 'field list'

# Tabela CDS

In [22]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    


In [23]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


In [24]:
num_list = 0                            # cdc ID que falta ver
for c in range(len(ID_cds)):
        cur.execute("""
                INSERT INTO CDS (ID, Translacion_seq, Localization, ID_protein, Protein)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                (ID + f"_{num_list}", str(Translacion_seq[num_list]), str(Localization[num_list]), str(ID_protein[num_list]), str(Protein[num_list]))) 
        num_list += 1

# Tabela References

In [25]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [26]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("None")

    
    n = re.search(r"title:\s+([\w\s')(-]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("None")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("None")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("None") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

/tmp/ipykernel_3961/713990561.py:26: FutureWarning: Possible set difference at position 20
  exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)


In [27]:
print(ID)
print(Pubmed)
print(Authors)
print(Title)
print(Journal)
print(Consortium)
print(Remark)

NC_003539
['1754400', 'None', 'None', 'None']
[['Puchta,H', 'Ramm,K', 'Luckinger,R', 'Hadas,R', 'Bar-Joseph,M.', 'Snger,H.L.'], ['Puchta,H', 'Ramm,K', 'Luckinger,R', 'Freimueller,K.', 'Saenger,H.L.'], [], ['Saenger,H.L.']]
['Primary and secondary structure of citrus viroid IV (CVd IV)', 'Citrus Viroid IV (CVd IV)', 'Direct Submission', 'Direct Submission']
['Nucleic Acids Res. 19 (23), 6640 (1991)', 'Unpublished', 'Submitted (09-MAY-2007) National Center for Biotechnology Information, NIH, Bethesda, MD 20894, USA', 'Submitted (09-MAR-1989) Saenger H.L., Max-Planck-Institut fuer Biochemie, Am Klopferspitz, D-8033 Martinsried bei Muenchen, FRG']
['None', 'None', 'NCBI Genome Project', 'None']
['None', 'None', 'None', 'None']


In [28]:
reference_count = 0                  # erro weird 
for c in Pubmed:    
        cur.execute("""
                INSERT INTO references (ID, Pubmed, ID_authors, Title, Journal, Consortium, Remark)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """,
                (ID + f"_{reference_count}", str(Pubmed[reference_count]), str(Authors[reference_count]), str(Title[reference_count]), str(Journal[reference_count]), str(Consortium[reference_count]), str(Remark[reference_count])))
        reference_count += 1

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'references (ID, Pubmed, ID_authors, Title, Journal, Consortium, Remark)
        ' at line 1

# Tabela Authors


In [29]:
for c in Authors:
        for i, a in enumerate(c):
                cur.execute("""
        INSERT INTO Authors (ID_authors, Names)    
        VALUES (%s, %s)
        """,
                (str(c) + f"_{i}", str(a)))

ProgrammingError: 1054 (42S22): Unknown column 'ID_authors' in 'field list'

# Tabela Pubmed_Info

In [30]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
for c in Pubmed:
    if c != "None":
        Entrez.email = 'sa.bruno.2001@gmail.com'
        # handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
        handle = Entrez.efetch(db='pubmed',retmode='xml',id=c)
        resultado = Entrez.read(handle)
        q = str(resultado["PubmedArticle"][0])
        
        if re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q):
            o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
            Abstract.append(o.group(1))
        else:
            Abstract.append("None")
        
        if re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q):
            p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
            DOI.append(p.group(1))
        else:
            DOI.append("None")

        if re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q):
            t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
            Affiliation.append(t.group(1))
        else:
            Affiliation.append("None")
    if c == "None":
            Abstract.append("None")
            DOI.append("None")
            Affiliation.append("None")



In [ ]:
print(Pubmed)
print(Title)
print(Abstract)
print(DOI)


['1754400', 'None', 'None', 'None']
['Primary and secondary structure of citrus viroid IV (CVd IV)', 'Citrus Viroid IV (CVd IV)', 'Direct Submission', 'Direct Submission']
['None', 'None', 'None', 'None']
['PMC329237', 'None', 'None', 'None']


In [ ]:
# try:
#         pubmed_count= 0
#         for c in Pubmed:
#                 cur.execute("""
#                         INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI)   
#                         VALUES (%s, %s, %s, %s)
#                         """,
#                         (Pubmed[pubmed_count] + f"_{pubmed_count}", Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count]))   
#         if c != "None":
#                 pubmed_count += 1
# except:
#         print("Erro Povoar Pubmed")

In [ ]:
pubmed_count= 0
for c in Pubmed :
        if c != "None":
                cur.execute("""
                        INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI, Affiliation)   
                        VALUES (%s, %s, %s, %s, %s)
                        """,
                        (Pubmed[pubmed_count], Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count], Affiliation[pubmed_count]))  
                pubmed_count += 1 
        else:
                pubmed_count += 1

# Lixo para baixo

In [ ]:
#     cur.execute("""
#         INSERT INTO Authors (ID_authors, Affiliation)    
#         VALUES (%s, %s, %s))
#         """,
#         (Authors, Affiliation))    